In [1]:
import os
import pandas as pd

In [85]:
from linearmodels import PanelOLS, RandomEffects, IV2SLS
from formulaic import model_matrix
import statsmodels.api as sm

In [3]:
MAIN = os.getcwd() + '/'
DATA = MAIN + "data/"

In [4]:
states = pd.read_csv(DATA+"vdem10.csv")

/var/folders/wk/syz2_hq141l5z1r4sy2xjj080000gn/T/ipykernel_39810/4003265948.py:1: DtypeWarning: Columns (363,364,365,393,409,829,830,1175,3002,3223,3224,3279,3281,3285,3286,3288,3289) have mixed types. Specify dtype option on import or set low_memory=False.
  states = pd.read_csv(DATA+"vdem10.csv")


In [80]:
small_states = states[["year", "country_name", "v2elsuffrage", "e_miurbpop"]]
small_states['year'] = small_states['year'].apply(lambda x: int(x))
small_states = small_states.dropna()
small_states.to_stata("states.dta", write_index=False)

/var/folders/wk/syz2_hq141l5z1r4sy2xjj080000gn/T/ipykernel_39810/2632593194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_states['year'] = small_states['year'].apply(lambda x: int(x))


In [81]:
small_states.head()

,year,country_name,v2elsuffrage,e_miurbpop
11,1800,Mexico,0.0,844.2
12,1801,Mexico,0.0,850.4
13,1802,Mexico,0.0,856.6
14,1803,Mexico,0.0,862.8
15,1804,Mexico,0.0,869.0


## OLS

In [67]:
y = "v2elsuffrage"
X = "e_miurbpop"
X = X.split()
model = y + " ~ " + " + ".join(X)
print(model)

v2elsuffrage ~ e_miurbpop


In [68]:
y, X = model_matrix(model, small_states)

In [69]:
y

,v2elsuffrage
11,0.0
12,0.0
13,0.0
14,0.0
15,0.0
...,...
25435,100.0
25436,100.0
25437,100.0
25438,100.0


In [70]:
X

,Intercept,e_miurbpop
11,1.0,844.20
12,1.0,850.40
13,1.0,856.60
14,1.0,862.80
15,1.0,869.00
...,...,...
25435,1.0,4004.72
25436,1.0,4024.04
25437,1.0,4043.36
25438,1.0,4062.68


In [71]:
ols_model = sm.OLS(y, X)
ols_results = ols_model.fit()
print(ols_results.summary())

                            OLS Regression Results                            
Dep. Variable:           v2elsuffrage   R-squared:                       0.056
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     1089.
Date:                Sat, 24 Sep 2022   Prob (F-statistic):          5.07e-232
Time:                        10:05:36   Log-Likelihood:                -94464.
No. Observations:               18273   AIC:                         1.889e+05
Df Residuals:                   18271   BIC:                         1.889e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     42.0479      0.336    125.239      0.0

### Equivalent to linearmodels IV2SLS

In [72]:
ols2_model = IV2SLS.from_formula(model, small_states)
ols2_results = ols2_model.fit(cov_type="unadjusted")
print(ols2_results)

                            OLS Estimation Summary                            
Dep. Variable:           v2elsuffrage   R-squared:                      0.0562
Estimator:                        OLS   Adj. R-squared:                 0.0562
No. Observations:               18273   F-statistic:                    1089.1
Date:                Sat, Sep 24 2022   P-value (F-stat)                0.0000
Time:                        10:05:36   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      42.048     0.3357     125.25     0.0000      41.390      42.706
e_miurbpop     0.0011  3.198e-05     33.002     0.00

## Fixed effects

### Equivalent to `xtset panelvar` in stata

In [75]:
small_states_fe = small_states
states_ids = pd.Categorical(small_states_fe.country_name)
small_states_fe = small_states_fe.set_index(["country_name","year"])
small_states_fe['country_name'] = states_ids
small_states_fe.head()

v2elsuffrage  e_miurbpop country_name
country_name year                                       
Mexico       1800           0.0       844.2       Mexico
             1801           0.0       850.4       Mexico
             1802           0.0       856.6       Mexico
             1803           0.0       862.8       Mexico
             1804           0.0       869.0       Mexico

In [77]:
exog_vars = ['e_miurbpop']
exog = sm.add_constant(small_states_fe[exog_vars])
panel_model = PanelOLS(small_states_fe.v2elsuffrage, exog, entity_effects=True).fit()
print(panel_model)

                          PanelOLS Estimation Summary                           
Dep. Variable:           v2elsuffrage   R-squared:                        0.1485
Estimator:                   PanelOLS   R-squared (Between):             -0.3293
No. Observations:               18273   R-squared (Within):               0.1485
Date:                Sat, Sep 24 2022   R-squared (Overall):              0.0091
Time:                        10:08:43   Log-likelihood                 -9.11e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3159.2
Entities:                         164   P-value                           0.0000
Avg Obs:                       111.42   Distribution:                 F(1,18108)
Min Obs:                       1.0000                                           
Max Obs:                       201.00   F-statistic (robust):             3159.2
                            

## Fixed effects with state-clustered SEs

In [84]:
exog_vars = ['e_miurbpop']
exog = sm.add_constant(small_states_fe[exog_vars])
panel_model = PanelOLS(small_states_fe.v2elsuffrage, exog, entity_effects=True).fit(cov_type="clustered", cluster_entity=True)
print(panel_model)

                          PanelOLS Estimation Summary                           
Dep. Variable:           v2elsuffrage   R-squared:                        0.1485
Estimator:                   PanelOLS   R-squared (Between):             -0.3293
No. Observations:               18273   R-squared (Within):               0.1485
Date:                Sat, Sep 24 2022   R-squared (Overall):              0.0091
Time:                        10:15:35   Log-likelihood                 -9.11e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3159.2
Entities:                         164   P-value                           0.0000
Avg Obs:                       111.42   Distribution:                 F(1,18108)
Min Obs:                       1.0000                                           
Max Obs:                       201.00   F-statistic (robust):             17.197
                            

## Random effects

In [88]:
re_model = RandomEffects(small_states_fe.v2elsuffrage, exog).fit()
print(re_model)

                        RandomEffects Estimation Summary                        
Dep. Variable:           v2elsuffrage   R-squared:                        0.1515
Estimator:              RandomEffects   R-squared (Between):             -0.1822
No. Observations:               18273   R-squared (Within):               0.1485
Date:                Sat, Sep 24 2022   R-squared (Overall):             -0.0246
Time:                        10:27:59   Log-likelihood                -9.119e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      3261.0
Entities:                         164   P-value                           0.0000
Avg Obs:                       111.42   Distribution:                 F(1,18271)
Min Obs:                       1.0000                                           
Max Obs:                       201.00   F-statistic (robust):             3120.2
                            